<a href="https://colab.research.google.com/github/SajalSinha/CardioVascular-Risk-Prediction/blob/main/CardiovascularRiskPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>